## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,San Cristobal,7.7669,-72.2250,86.00,66,40,11.50,VE,scattered clouds
1,1,San Quintin,30.4833,-115.9500,65.50,62,40,15.14,MX,scattered clouds
2,2,Qaanaaq,77.4840,-69.3632,32.54,92,100,5.50,GL,overcast clouds
3,3,Rikitea,-23.1203,-134.9692,75.04,75,94,12.88,PF,light rain
4,4,Vila Do Maio,15.1333,-23.2167,79.59,84,85,18.54,CV,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
## We will follow 6.5.3 (2021).
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
## We refactor code from 6.5.3 (2021).
city_temperature_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"]>= min_temp)]
city_temperature_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,San Cristobal,7.7669,-72.2250,86.00,66,40,11.50,VE,scattered clouds
3,3,Rikitea,-23.1203,-134.9692,75.04,75,94,12.88,PF,light rain
4,4,Vila Do Maio,15.1333,-23.2167,79.59,84,85,18.54,CV,overcast clouds
8,8,San Patricio,28.0170,-97.5169,89.69,56,18,14.99,US,few clouds
11,11,Atuona,-9.8000,-139.0333,79.95,61,2,8.01,PF,clear sky
...,...,...,...,...,...,...,...,...,...,...
654,654,Auki,-8.7676,160.7034,75.88,89,17,3.18,SB,few clouds
655,655,Uruzgan,32.8333,66.0000,75.47,12,0,5.77,AF,clear sky
658,658,Yanam,16.7333,82.2167,85.26,78,1,4.47,IN,clear sky
664,664,Gumdag,39.2061,54.5906,88.00,24,5,13.04,TM,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
city_temperature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 0 to 666
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              222 non-null    int64  
 1   City                 222 non-null    object 
 2   Lat                  222 non-null    float64
 3   Lng                  222 non-null    float64
 4   Max Temp             222 non-null    float64
 5   Humidity             222 non-null    int64  
 6   Cloudiness           222 non-null    int64  
 7   Wind Speed           222 non-null    float64
 8   Country              222 non-null    object 
 9   Current Description  222 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 19.1+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## Our new dataframe has 222 rows of data, and the info() function indicates a total of
## 222 non-null rows. Hence, there are no empty rows for us to drop.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_temperature_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Cristobal,VE,86.00,scattered clouds,7.7669,-72.2250,
3,Rikitea,PF,75.04,light rain,-23.1203,-134.9692,
4,Vila Do Maio,CV,79.59,overcast clouds,15.1333,-23.2167,
8,San Patricio,US,89.69,few clouds,28.0170,-97.5169,
11,Atuona,PF,79.95,clear sky,-9.8000,-139.0333,
15,Kapaa,US,79.20,clear sky,22.0752,-159.3190,
18,Bubaque,GW,78.06,scattered clouds,11.2833,-15.8333,
21,Vila,PT,75.74,few clouds,42.0304,-8.1588,
26,Starkville,US,75.99,clear sky,33.4504,-88.8184,
28,Butaritari,KI,82.85,overcast clouds,3.0707,172.7902,


In [8]:
## For this block of code, we will be referencing 6.5.4 (2021).
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Grab the first hotel from the results and store the name.
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
## Checking our Hotel DataFrame.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Cristobal,VE,86.00,scattered clouds,7.7669,-72.2250,Posada Villaven C.A.
3,Rikitea,PF,75.04,light rain,-23.1203,-134.9692,Pension Maro'i
4,Vila Do Maio,CV,79.59,overcast clouds,15.1333,-23.2167,Hotel Marilu
8,San Patricio,US,89.69,few clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
11,Atuona,PF,79.95,clear sky,-9.8000,-139.0333,Villa Enata
15,Kapaa,US,79.20,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Bubaque,GW,78.06,scattered clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
21,Vila,PT,75.74,few clouds,42.0304,-8.1588,Hotel Castrum Villae by Walk Hotels
26,Starkville,US,75.99,clear sky,33.4504,-88.8184,Hilton Garden Inn Starkville
28,Butaritari,KI,82.85,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


In [10]:
# 7. Drop the rows where there is no Hotel Name.
## As seen in 4.5.2 (2021).
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Cristobal,VE,86.00,scattered clouds,7.7669,-72.2250,Posada Villaven C.A.
3,Rikitea,PF,75.04,light rain,-23.1203,-134.9692,Pension Maro'i
4,Vila Do Maio,CV,79.59,overcast clouds,15.1333,-23.2167,Hotel Marilu
8,San Patricio,US,89.69,few clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
11,Atuona,PF,79.95,clear sky,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
654,Auki,SB,75.88,few clouds,-8.7676,160.7034,Auki Motel
655,Uruzgan,AF,75.47,clear sky,32.8333,66.0000,
658,Yanam,IN,85.26,clear sky,16.7333,82.2167,The Regency Hotel
664,Gumdag,TM,88.00,clear sky,39.2061,54.5906,


In [11]:
# 8a. Create the output File (CSV)
## Since we are in Vacation_Search directory, we can define our filepath with only the csv filename.
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
## We follow the HTML code format in 6.5.4 (2021) so that it fits our needs for the
## Module 6 Challenge, namely by showing city, country, weather description, and max temperature.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd> 
<dt>Current Description</dt><dd>{Current Description}</dd> 
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
## We will use code from 6.5.4 (2021), though for Module 6 Challenge (2021),
## we don't need to add a heat map.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))